<a href="https://colab.research.google.com/github/bonghoa15/gee/blob/master/download_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is assumed that you are authenticated with Google Earth engine your account. If not, sign up your email with GEE with link:  https://earthengine.google.com/new_signup/

In [2]:
!pip install google-api-python-client

In [0]:
import ee

In [4]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/xgHIWR-kwd_epbjFFEKFcde5tqYgale8OXrNoDhuRDr6o0g61M8Y13A

Successfully saved authorization token.


In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
ls: cannot open directory '.': Transport endpoint is not connected


In [0]:
# %cd 'gdrive/My Drive'

In [0]:
gFolder = 'ee' # create folder if not exist

In [0]:
#border Nigeria
border = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
                    .filterMetadata("country_na", "equals", "Nigeria").geometry()

In [0]:
#image/imagecollection
nightlight = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
                                    .select(['avg_rad'])\
                                    .sort('system:time_end', False)\
                                    .limit(12) #most recent 12 months
worldpop = ee.ImageCollection("WorldPop/GP/100m/pop")\
                        .filterMetadata("country", "equals", "NGA")\
                        .filterMetadata("year", "equals", 2019) #year 2019

In [0]:
#download image collection
def download_imgcol(imgcol, folder, border = border, aggfunc = 'median', band,scale = 100):
    # collection metadata
    imagename = imgcol.getInfo()["id"]
    id = f"{imagename}_{band}_{aggfunc}"

    # first image metadata
    meta = imgcol.first().clip(border).getInfo()["bands"][0]
    crs_transform = str(meta["crs_transform"])
    dimensions = meta["dimensions"]

    # clip border, select one band 
    image = getattr(imgcol, aggfunc)().clip(border)
    image = image.set(dict(id=id, crs_transform=crs_transform))
    image = image.select(band)

    #getinfo
    meta = image.getInfo()
    props = meta["properties"]
    crs_transformation = props.get('crs_transform')
    crs = meta['bands'][0]['crs']

    #download 
    task = ee.batch.Export.image.toDrive(image = image,
                                         folder = folder,
                                         description = id,
                                         scale= scale, 
                                         fileFormat= 'GeoTIFF',
                                         maxPixels=int(1e10));
                      
    task.start()
    return print(task.status())

In [0]:
download_imgcol(nightlight,folder = gFolder, band = 'avg_band')

In [0]:
download_imgcol(worldpop, folder = gFolder, band = 'population')